In [68]:
%load_ext autoreload
%autoreload 2
from functools import partial

import keras

from datasets import example_datasets, to_numpy
from models import mixture_poissons,location_specific_linear, CustomPenalizedMixtureDecisionModel
from metrics import mixture_poi_loss, get_bpr_loss_func, mix_bpr, get_penalized_bpr_loss_func_mix, cross_ratio_decision, get_perturbed_bpr_func
from experiments import training_loop, training_loop_score_function_trick, score_function_trick, overall_gradient_calculation
from plotting_funcs import plot_losses, plot_frontier

import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
seed=360
num_components=4
learning_rate = 0.005
epochs=500
outdir = '/cluster/home/kheuto01/testdir'
penalty = 5000
threshold = 0.55
K=4
do_only=True
# tracts/distributions
S=12
# history/features
H = 3
# total timepoints
T= 500
perturbed_sigma=0.1

In [70]:
train_dataset, val_dataset, test_dataset = example_datasets(H, T, seed=seed)
train_X_THS, train_y_TS = to_numpy(train_dataset)
val_X_THS, val_y_TS = to_numpy(val_dataset)
input_shape = (H,S)

bpr_K = get_perturbed_bpr_func(K, sigma=perturbed_sigma)

In [73]:
model = CustomPenalizedMixtureDecisionModel(num_features=H, num_locations=S,
                 member_model=location_specific_linear,
                 member_distribution=tfp.distributions.Poisson,
                 decision_func=cross_ratio_decision,
                 bpr_func=bpr_K,
                 bpr_threshold=threshold,
                 penalty=penalty,
                 objective_includes_likelihood=True,
                 objective_includes_bpr=True,
                 num_components=4,
                 num_score_func_samples=50,
                 seed=360,)

optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
#model.compile(optimizer=optimizer)

In [74]:
for step, (x_BHS, y_BS) in enumerate(train_dataset):

    with tf.GradientTape() as jacobian_tape, tf.GradientTape() as loss_tape:
        mixture_distribution = model._get_mixture_distribution(x_BHS)
        sample_y_MBS = mixture_distribution.sample(model.num_score_func_samples)

        sample_log_likelihood_MBS = mixture_distribution.log_prob(sample_y_MBS)

        sample_decisions_MBS = model.decision_func(sample_y_MBS)
        expected_decisions_BS = tf.reduce_mean(sample_decisions_MBS, axis=0)

        loss_B, metrics = model.calc_loss_and_metrics(y_BS, mixture_distribution, expected_decisions_BS)


    # The lowercase "p" signifies that these are lists of length P, P = number of trainable variables
    jacobian_pMBS = jacobian_tape.jacobian(sample_log_likelihood_MBS, model.trainable_variables)
    param_gradient_pBS = score_function_trick(jacobian_pMBS, sample_decisions_MBS)
    loss_gradients_BS = loss_tape.gradient(loss_B, expected_decisions_BS)
    overall_gradient_p = [overall_gradient_calculation(g, loss_gradients_BS) for g in param_gradient_pBS]
    optimizer.apply_gradients(overall_gradient_p, model.trainable_variables)

TypeError: You are passing KerasTensor(type_spec=TensorSpec(shape=(50, 300, 12), dtype=tf.float32, name=None), name='tf.reshape_32/Reshape:0', description="created by layer 'tf.reshape_32'"), an intermediate Keras symbolic input/output, to a TF API that does not allow registering custom dispatchers, such as `tf.cond`, `tf.function`, gradient tapes, or `tf.map_fn`. Keras Functional model construction only supports TF API calls that *do* support dispatching, such as `tf.math.add` or `tf.reshape`. Other APIs cannot be called directly on symbolic Kerasinputs/outputs. You can work around this limitation by putting the operation in a custom Keras layer `call` and calling that layer on this symbolic input/output.

In [79]:
sample_y_MBS.to_numpy()

AttributeError: 'KerasTensor' object has no attribute 'to_numpy'

In [57]:
mix_dist

<tfp.distributions.MixtureSameFamily 'MixtureSameFamily' batch_shape=[300, 12] event_shape=[] dtype=float32>

In [58]:
model.num_score_func_samples 

50

In [59]:
mix_dist.sample(model.num_score_func_samples)

<KerasTensor: shape=(50, 300, 12) dtype=float32 (created by layer 'tf.reshape_18')>

In [60]:
x_batch_train

<tf.Tensor: shape=(300, 3, 12), dtype=float32, numpy=
array([[[  7.,   7.,   7., ...,   0., 100.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ..., 100.,   0.,   0.]],

       [[  7.,   7.,   7., ...,   0.,   0., 100.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.]],

       [[  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ..., 100.,   0.,   0.]],

       ...,

       [[  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.]],

       [[  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.]],

       [[  7.,   7.,   7., ..., 100.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.]]], dty

In [49]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 3, 12), dtype=tf.float32, name=None), TensorSpec(shape=(None, 12), dtype=tf.float32, name=None))>

In [54]:
train_X_THS

array([[[  7.,   7.,   7., ..., 100.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.]],

       [[  7.,   7.,   7., ...,   0., 100.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.]],

       [[  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.]],

       ...,

       [[  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.]],

       [[  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.]],

       [[  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.],
        [  7.,   7.,   7., ...,   0.,   0.,   0.]]], dtype=float32)

In [67]:
model.fit(train_dataset, epochs=5, verbose=1, batch_size=300)

Epoch 1/5


> /cluster/home/kheuto01/code/prob_diff_topk/models.py(210)train_step()
    208             mixture_distribution = self._get_mixture_distribution(x_BHS)
    209             import pdb;pdb.set_trace();
--> 210             sample_y_MBS = mixture_distribution.sample(self.num_score_func_samples)
    211 
    212             sample_log_likelihood_MBS = mixture_distribution.log_prob(sample_y_MBS)

<tf.Tensor 'IteratorGetNext:0' shape=(None, 3, 12) dtype=float32>
<tf.Tensor 'IteratorGetNext:0' shape=(None, 3, 12) dtype=float32>


TypeError: in user code:

    File "/cluster/tufts/hugheslab/kheuto01/mambaforge/envs/pip_k2_tf/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/cluster/tufts/hugheslab/kheuto01/mambaforge/envs/pip_k2_tf/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/cluster/tufts/hugheslab/kheuto01/mambaforge/envs/pip_k2_tf/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/cluster/home/kheuto01/code/prob_diff_topk/models.py", line 210, in train_step
        sample_y_MBS = mixture_distribution.sample(self.num_score_func_samples)
    File "/cluster/tufts/hugheslab/kheuto01/mambaforge/envs/pip_k2_tf/lib/python3.11/site-packages/tensorflow_probability/python/distributions/distribution.py", line 

In [51]:
    train_X_THS, train_y_TS = to_numpy(train_dataset)
    val_X_THS, val_y_TS = to_numpy(val_dataset)

In [35]:
model.compile(optimizer=optimizer)

In [6]:
model=mix_model_penalized

In [7]:
for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
    with tf.GradientTape() as jacobian_tape, tf.GradientTape() as loss_tape:
                prob_params_BSK, mixture_weights_KS = model(x_batch_train, training=True)

In [9]:
mixture_weights_SK = tf.transpose(mixture_weights_KS, perm=[1,0])

In [11]:
mix = tfp.distributions.MixtureSameFamily(
    mixture_distribution=tfp.distributions.Categorical(probs=mixture_weights_SK),
    components_distribution = tfp.distributions.Poisson(rate=prob_params_BSK))

In [20]:
sample_y_MBS = mix.sample(2)


In [21]:
log_probs_MBS = mix.log_prob(sample_y_MBS)

In [19]:
log_probs_MBS

<tf.Tensor: shape=(2, 300, 12), dtype=float32, numpy=
array([[[-0.70101863, -3.49464   , -0.6986769 , ..., -0.6931472 ,
         -0.6931473 , -1.0596601 ],
        [-3.5308998 , -4.944219  , -0.6986769 , ..., -4.9409027 ,
         -1.0596601 , -0.6931472 ],
        [-4.0951333 , -0.7111152 , -3.7875445 , ..., -0.6931472 ,
         -0.6981911 , -0.6931472 ],
        ...,
        [-0.70101863, -3.4190197 , -0.6986769 , ..., -0.6931472 ,
         -0.6931473 , -0.6931472 ],
        [-3.981664  , -4.2890096 , -0.6986769 , ..., -1.0596602 ,
         -1.3627218 , -1.0596601 ],
        [-4.0275745 , -0.7111152 , -3.8916945 , ..., -2.1193204 ,
         -6.206031  , -1.0596601 ]],

       [[-0.70101863, -0.7111152 , -0.6986769 , ..., -0.6931472 ,
         -0.6931473 , -2.1193204 ],
        [-3.9768908 , -3.4446435 , -4.548599  , ..., -5.537909  ,
         -0.6931473 , -0.6931472 ],
        [-0.70101863, -0.7111152 , -0.6986769 , ..., -0.6931472 ,
         -4.6353803 , -2.1193204 ],
        ...,


In [16]:
mix.mean()

<tf.Tensor: shape=(300, 12), dtype=float32, numpy=
array([[ 7.8314114,  7.820159 ,  7.870078 , ...,  0.6931473,  0.6931472,
         0.6931472],
       [ 7.8314114,  7.820159 ,  7.870078 , ..., 31.709133 ,  0.6931472,
         0.6931472],
       [ 7.8314114,  7.820159 ,  7.870078 , ...,  0.6931473, 31.864498 ,
         0.6931472],
       ...,
       [ 7.8314114,  7.820159 ,  7.870078 , ...,  0.6931473,  0.6931472,
         0.6931472],
       [ 7.8314114,  7.820159 ,  7.870078 , ...,  0.6931473, 48.643055 ,
         0.6931472],
       [ 7.8314114,  7.820159 ,  7.870078 , ...,  0.6931473, 31.864498 ,
         0.6931472]], dtype=float32)>

In [22]:
losses_penalized = training_loop_score_function_trick(mix_model_penalized, penalized_bpr_loss, optimizer,
                                    epochs, train_dataset, val_dataset, negative_bpr_K,
                                    cross_ratio_decision,
                                    num_score_func_samples=10,
                                    component_likelihood=tfp.distributions.Poisson,
                                    verbose=True)

OperatorNotAllowedInGraphError: in user code:

    File "/cluster/home/kheuto01/code/prob_diff_topk/metrics.py", line 150, in uncurried_penalized_mix_bpr  *
        mixture_bpr_loss_val = mix_bpr(y_true, y_pred, negative_bpr_K_func=negative_bpr_K_func)
    File "/cluster/home/kheuto01/code/prob_diff_topk/metrics.py", line 138, in mix_bpr  *
        component_preds, mixture_weights = y_pred

    OperatorNotAllowedInGraphError: Iterating over a symbolic `tf.Tensor` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating this function with @tf.function. If that does not work, then you may be using an unsupported feature or your source code may not be visible to AutoGraph. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md#access-to-source-code for more in